In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import subprocess
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc, roc_curve, confusion_matrix, classification_report
# seed = 2024
# # 固定种子，结果可重复
# random.seed(seed)
# np.random.seed(seed)

In [2]:
data = pd.read_excel('/Users/rachael/Downloads/train_df_labelled copy.xlsx')
#the data verified by the third person will be used as the train set finally
file =  data[['content','sentiment']]
file['sentiment'].value_counts()
label_sentiment= file['sentiment'].value_counts()
label_proportions_sentiment= file['sentiment'].value_counts(normalize=True)
print("Counts of each label_sentiment:")
print(label_sentiment)
print("\nProportions of each label_sentiment:")
print(label_proportions_sentiment)

Counts of each label_sentiment:
sentiment
Negative    262
Neither      99
Positive     48
Name: count, dtype: int64

Proportions of each label_sentiment:
sentiment
Negative    0.640587
Neither     0.242054
Positive    0.117359
Name: proportion, dtype: float64


In [3]:
import re
def remove_urls(text):
    # 定义URL的正则表达式
    url_pattern = r'https?://\S+|www\.\S+'
    # 使用re.sub()函数替换URL为空字符串
    no_url_text = re.sub(url_pattern, '', text)
    return no_url_text

def clean_text(text):
    # Remove anything that is not a letter or space
    clean_text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Optional: Convert text to lower case
    clean_text = clean_text.lower()
    return clean_text

In [4]:
file['content'] = file['content'].apply(remove_urls) 
file['content'] = file['content'].apply(clean_text) 
file['sentiment'] = file['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})
y = file['sentiment']
X = file['content']

/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_6369/3949177293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(remove_urls)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_6369/3949177293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(clean_text)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_6369/3949177293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [5]:
from gensim.models import KeyedVectors
import spacy
nlp = spacy.load("nl_core_news_sm")
#tokenize
def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

#load pre-trained word2vec embedding 
model_path = '/Users/rachael/Downloads/39/model.bin'
word_vectors = KeyedVectors.load_word2vec_format(model_path, binary=True)
vector_size = word_vectors.vector_size #dimension of embedding 100

# word embedding mapping and convert sentences to the average of vectors
def sentence_to_avg_vec(tokens, model, vector_size):
    vec = np.zeros(vector_size)
    count = 0
    for token in tokens:
        if token in model:
            vec += model[token]
            count += 1
    if count != 0:
        vec /= count
    return vec

In [6]:
# Initialize parameters
best_score = 0
best_params = {}

#hyperparameter range
param_dist = {
    'C': np.logspace(-4, 4, 20),  # Inverse of regularization strength
    'solver': ['newton-cg', 'lbfgs', 'saga'],  # Solvers that support multinomial loss
    'max_iter': [20000, 30000,40000],  # Maximum number of iterations
}

test_scores = []
best_params_list = []
f1_scores = []

In [8]:
# Stratified K-fold for maintaining label distribution,shuffle=True确保每次迭代的数据划分不同
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

#define model
lr = LogisticRegression(multi_class='multinomial', random_state=42,class_weight='balanced')
randomized_search = RandomizedSearchCV(
        estimator=lr,
        param_distributions=param_dist,
        n_iter=100,  # Number of parameter settings that are sampled
        scoring='f1_macro',
        n_jobs=-1,  # Use all available cores
        cv=inner_cv,
        random_state=42,
        verbose=1
    )
# Loop over each fold
for train_index, test_index in outer_cv.split(X, y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainy, testy = y.iloc[train_index], y.iloc[test_index]
    
    # convert y_train and y_test to DataFrame
    y_train_df = pd.DataFrame(trainy).reset_index(drop=True)
    y_test_df = pd.DataFrame(testy).reset_index(drop=True)

    # concat X_train and y_train , X_test and y_test
    train_df = pd.concat([trainX.reset_index(drop=True), y_train_df], axis=1)
    test_df = pd.concat([testX.reset_index(drop=True), y_test_df], axis=1)
    
    #process the data to the format that is suitable for data augmentation
    train_df['content'] = train_df['content'].apply(lambda x: ' '.join(x.split()))
    sentiment_0 = train_df[train_df['sentiment'] == 0]
    sentiment_1 = train_df[train_df['sentiment'] == 1]
    sentiment_2 = train_df[train_df['sentiment'] == 2]
    
    #expand class0
    output_file_path0 = '/Users/rachael/Desktop/data/sentiment_0.txt'
    #convert DataFrame data into a simple tab-separated text file format，required by the data augmentation technique
    with open(output_file_path0, 'w', encoding='utf-8') as f:
        for _, row in sentiment_0.iterrows():
            f.write(f"{row['sentiment']}\t{row['content']}\n")
    subprocess.run([
    'python', '/Users/rachael/Desktop/data/augment.py',
    '--input', output_file_path0,
    '--output', '/Users/rachael/Desktop/data/augmented_train0.txt',  # Output as txt if required
    '--num_aug', '9',
    '--alpha_sr', '0.1',
    '--alpha_rd', '0.1',
    '--alpha_ri', '0.1',
    '--alpha_rs', '0.1'
])

    #expand class1
    output_file_path1 = '/Users/rachael/Desktop/data/sentiment_1.txt'
    with open(output_file_path1, 'w', encoding='utf-8') as f:
        for _, row in sentiment_1.iterrows():
            f.write(f"{row['sentiment']}\t{row['content']}\n")      
    subprocess.run([
    'python', '/Users/rachael/Desktop/data/augment.py',
    '--input', output_file_path1,
    '--output', '/Users/rachael/Desktop/data/augmented_train1.txt',  # Output as txt if required
    '--num_aug', '1',
    '--alpha_sr', '0.1',
    '--alpha_rd', '0.1',
    '--alpha_ri', '0.1',
    '--alpha_rs', '0.1'
])
    
    #expand class2
    output_file_path2 = '/Users/rachael/Desktop/data/sentiment_2.txt'
    with open(output_file_path2, 'w', encoding='utf-8') as f:
        for _, row in sentiment_2.iterrows():
            f.write(f"{row['sentiment']}\t{row['content']}\n")     
    subprocess.run([
    'python', '/Users/rachael/Desktop/data/augment.py',
    '--input', output_file_path2,
    '--output', '/Users/rachael/Desktop/data/augmented_train2.txt',  # Output as txt if required
    '--num_aug', '4',
    '--alpha_sr', '0.1',
    '--alpha_rd', '0.1',
    '--alpha_ri', '0.1',
    '--alpha_rs', '0.1'
])
    
    # Load augmented data
    file4 = pd.read_csv('/Users/rachael/Desktop/data/augmented_train0.txt', delimiter='\t', header=None, names=['sentiment', 'content'])
    file5 = pd.read_csv('/Users/rachael/Desktop/data/augmented_train1.txt', delimiter='\t', header=None, names=['sentiment', 'content'])
    file6 = pd.read_csv('/Users/rachael/Desktop/data/augmented_train2.txt', delimiter='\t', header=None, names=['sentiment', 'content'])
    augmented_train = pd.concat([file4,file5,file6])

    #process train data
    augmented_train['content'] = augmented_train['content'].astype(str)
    y_train = augmented_train['sentiment']
    X_train = augmented_train['content']
    X_train_tokenized = X_train.apply(tokenize)
    X_train_embeddings = np.array([sentence_to_avg_vec(tokens, word_vectors, vector_size) for tokens in X_train_tokenized])
    #X_train_embeddings = scaler.fit_transform(X_train_embeddings1)

    #prepare test data
    test_df['content'] = test_df['content'].astype(str)
    y_test = test_df['sentiment']
    X_test = test_df['content']
    X_test_tokenized = X_test.apply(tokenize)
    X_test_embeddings = np.array([sentence_to_avg_vec(tokens, word_vectors, vector_size) for tokens in X_test_tokenized])
    #X_test_embeddings = scaler.transform(X_test_embeddings1)

    #train model 
    randomized_search.fit(X_train_embeddings, y_train)

    # Get the best model from Randomized Search
    best_model = randomized_search.best_estimator_
    best_params = randomized_search.best_params_
    best_score = randomized_search.best_score_

    # Evaluate on the test set
    test_predictions = best_model.predict(X_test_embeddings)
    test_f1_score = f1_score(y_test, test_predictions, average='macro')

    f1_scores.append(test_f1_score)
    best_params_list.append(best_params)

    print(f"Fold F1-score: {test_f1_score}")
    print(f"Best parameters: {best_params}")

mean_f1_score = np.mean(f1_scores)
std_f1_score = np.std(f1_scores)

print(f"Mean F1-score: {mean_f1_score}")
print(f"Standard deviation of F1-scores: {std_f1_score}")
print("Best parameters for each fold:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i + 1}: {params}")

generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_0.txt to /Users/rachael/Desktop/data/augmented_train0.txt with num_aug=9
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_1.txt to /Users/rachael/Desktop/data/augmented_train1.txt with num_aug=1
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_2.txt to /Users/rachael/Desktop/data/augmented_train2.txt with num_aug=4
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fold F1-score: 0.44674329501915705
Best parameters: {'solver': 'lbfgs', 'max_iter': 20000, 'C': 10000.0}
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_0.txt to /Users/rachael/Desktop/data/augmented_train0.txt with num_aug=9
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_1.txt to /Users/rachael/Desktop/data/augmented_train1.txt with num_aug=1
generated augmented sentences with eda for /Users/rachael/Desk

In [9]:
from collections import Counter
best_params_counter = Counter(tuple(sorted(params.items())) for params in best_params_list)
most_common_params = dict(best_params_counter.most_common(1)[0][0])

print("Most common best parameters:", most_common_params)

Most common best parameters: {'C': 10000.0, 'max_iter': 20000, 'solver': 'lbfgs'}


In [6]:
most_common_params = {'solver': 'lbfgs', 'max_iter': 40000, 'C': 206.913808111479}


In [7]:
X_sentences = [item for item in list(X)]
X_tokenized = X.apply(tokenize)
X_embeddings = np.array([sentence_to_avg_vec(tokens, word_vectors, 100) for tokens in X_tokenized])
#X_embeddings = scaler.fit_transform(X_embeddings)

final_model = LogisticRegression(multi_class='multinomial', random_state=42, **most_common_params,class_weight='balanced')
final_model.fit(X_embeddings, y)

LogisticRegression(C=206.913808111479, class_weight='balanced', max_iter=40000,
                   multi_class='multinomial', random_state=42)

In [8]:
test = pd.read_excel('/Users/rachael/Downloads/test_df_labelled.xlsx')
#process test data 
test = test[['content','sentiment']]
test['content'] = test['content'].apply(remove_urls) 
test['content'] = test['content'].apply(clean_text) 
test['sentiment'] = test['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})
test_tokenized = test['content'].apply(tokenize)
test_embedding = np.array([sentence_to_avg_vec(tokens,word_vectors,100) for tokens in test_tokenized])

#predict 
test_predictions = final_model.predict(test_embedding)
test_score = accuracy_score(test['sentiment'], test_predictions)
conf_matrix = confusion_matrix(test['sentiment'], test_predictions)
class_report = classification_report(test['sentiment'], test_predictions)
print(class_report)

              precision    recall  f1-score   support

           0       0.11      0.21      0.15        38
           1       0.68      0.60      0.63       245
           2       0.42      0.41      0.42       124

    accuracy                           0.50       407
   macro avg       0.40      0.41      0.40       407
weighted avg       0.55      0.50      0.52       407

